### New RS

In [1]:
class NewsRS:
    pass

__read_df__ 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('shared_articles.csv')

In [7]:
df.drop(columns='Unnamed: 0', inplace=True)

In [8]:
df

,url,title
0,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact..."
1,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact..."
2,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...
3,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour
4,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc..."
...,...,...
3117,https://startupi.com.br/2017/02/liga-ventures-...,"Conheça a Liga IoT, plataforma de inovação abe..."
3118,https://thenextweb.com/apps/2017/02/14/amazon-...,Amazon takes on Skype and GoToMeeting with its...
3119,https://code.org/about/2016,Code.org 2016 Annual Report
3120,https://www.bloomberg.com/news/articles/2017-0...,JPMorgan Software Does in Seconds What Took La...


Null_Values & Duplicated

In [9]:
df.isna().sum()

url      0
title    0
dtype: int64

In [11]:
df.duplicated().sum()

90

In [12]:
df.drop_duplicates(inplace=True)

In [76]:
df.reset_index(drop=True, inplace=True)

__pre_processing__ 

In [14]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [13]:
"Like"
"like"
"LIKE".lower()

'like'

In [28]:
en_stopewords = stopwords.words('English')

In [30]:
stemmer = PorterStemmer()

In [35]:
verb1 = 'Playing'
verb2 = 'Plays'
verb3 = 'Played'

stemmer.stem(verb1)

'play'

In [36]:
def pre_process(article):
    article = article.lower()
    clean_article = re.sub('[^a-z1-9 ]', '', article).strip()
    
    tokens = word_tokenize(clean_article)
    temp = []
    
    for token in tokens:
        if token not in en_stopewords:
            stemed_token = stemmer.stem(token)
            temp.append(stemed_token)
    
    return ' '.join(temp)

In [40]:
s = "I hate *& % laba $ plays on"
pre_process(s)

'hate laba play'

__Clean titles__

In [51]:
X = df.title.apply(pre_process)
X

0       ethereum virtual currenc enabl transact rival ...
2         bitcoin futur gbpcoin branson win usdcoin trump
3                               googl data center 36 tour
4            ibm want evolv internet blockchain technolog
5       ieee talk blockchain cloud comput oxfordcon co...
                              ...                        
3117    conhea liga iot plataforma de inovao aberta qu...
3118    amazon take skype gotomeet chime video confere...
3119                            codeorg 216 annual report
3120          jpmorgan softwar second took lawyer 36 hour
3121                              217 acquia partner year
Name: title, Length: 3032, dtype: object

__Feature_extraction__

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [53]:
tf_idf = TfidfVectorizer()

In [54]:
X_trans = tf_idf.fit_transform(X).toarray()

In [55]:
X_trans.shape

(3032, 5908)

### Model

In [48]:
from sklearn.neighbors import NearestNeighbors

In [83]:
def train():
    model = NearestNeighbors()
    model.fit(X_trans)

In [84]:
train()

In [68]:
def predict(article):
    clean_article = pre_process(article)
    clean_article_trans = tf_idf.transform([clean_article])
    indices = model.kneighbors(clean_article_trans, n_neighbors=5, return_distance=False)[0]
    return indices

In [69]:
article_to_reco = df.title.loc[0]

In [70]:
article_to_reco

"Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's"

In [71]:
predict(article_to_reco)

array([   0,  179,  176,  935, 2378], dtype=int64)

In [78]:
articles = df.loc[[   0,  179,  176,  935, 2378]]

In [79]:
articles

,url,title
0,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact..."
179,https://news.bitcoin.com/microsoft-embrace-dig...,Microsoft Continues to Embrace Ethereum & Bitc...
176,http://www.newsbtc.com/2016/04/02/ethereum-bit...,Ethereum and Bitcoin Are Market Leaders But No...
935,http://techcrunch.com/2016/05/16/blockchain-op...,"Blockchain open sources Thunder network, pavin..."
2378,https://www.cryptocoinsnews.com/bitcoin-accept...,Bitcoin Accepted! German Energy Giant Enables ...


In [81]:
for title in articles.title:
    print(title)

Ethereum, a Virtual Currency, Enables Transactions That Rival Bitcoin's
Microsoft Continues to Embrace Ethereum & Bitcoin - Bitcoin News
Ethereum and Bitcoin Are Market Leaders But Not Competitors
Blockchain open sources Thunder network, paving the way for instant bitcoin transactions
Bitcoin Accepted! German Energy Giant Enables Payments - CCN: Financial Bitcoin & Cryptocurrency News


In [91]:
article_to_reco = df.title.loc[173]

In [92]:
article_to_reco

'Venture Capitalists Inject Nearly Half a Billion in Bitcoin'

In [93]:
indices = predict(article_to_reco)

In [94]:
df.loc[indices]

,url,title
173,http://www.newsbtc.com/2016/04/04/venture-capi...,Venture Capitalists Inject Nearly Half a Billi...
2104,https://www.mindmeister.com/pt/694609910/bitco...,Bitcoin / Blockchain
2506,http://martinfowler.com/articles/injection.html,Inversion of Control Containers and the Depend...
543,http://www.npr.org/sections/health-shots/2016/...,Half Your Brain Stands Guard When Sleeping In ...
179,https://news.bitcoin.com/microsoft-embrace-dig...,Microsoft Continues to Embrace Ethereum & Bitc...
